In [38]:
import sys
import os
import getpass
import time
import itertools as itt

import nrl.model
import igraph
from tqdm import tqdm_notebook as tqdm
from pybel.dsl import ComplexAbundance
import networkx as nx
import bio2bel_hippie

In [4]:
print(sys.version)

3.7.0 (default, Aug  7 2018, 14:49:37) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-28)]


In [5]:
print(time.asctime())

Mon May 20 12:37:51 2019


In [7]:
print(getpass.getuser())

choyt


In [12]:
hippie_manager = bio2bel_hippie.Manager()
hippie_manager

<HippieManager url=sqlite:////home/choyt/.bio2bel/bio2bel.db>

In [13]:
if not hippie_manager.is_populated():
    hippie_manager.populate()

get uniprot mappings


/home/choyt/.virtualenvs/hbp/lib/python3.7/site-packages/bio2bel/manager/abstract_manager.py:38: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  cls._populate_original(self, *populate_args, **populate_kwargs)


get hippie


100%|██████████| 411430/411430 [00:47<00:00, 8673.19it/s]


In [14]:
bel_graph = hippie_manager.to_bel()

interactions: 100%|██████████| 411430/411430 [02:26<00:00, 2807.32it/s]


In [33]:
nx_graph = nx.Graph(
    (f'uniprot:{l.name}', f'uniprot:{r.name}')
    for node in tqdm(bel_graph)
    if isinstance(node, ComplexAbundance)
    for l, r in itt.combinations(node.members, 2)
)

In [35]:
nx.write_gml(nx_graph, 'edges.gml')
graph = igraph.Graph.Read_GML('edges.gml')

In [39]:
deep_walk_model = nrl.model.DeepWalkModel()
deep_walk_model

In [40]:
word2vec_model = deep_walk_model.fit(graph)

In [41]:
word2vec_model